# Ant-Fungi Hybrid Optimization Algorithm

Ant Colony Optimization (ACO) algorithm to incorporate some of the efficient path-finding strategies used by fungi (specifically slime mold or the mycelium network). This hybrid approach could leverage the speed of ants and the adaptive resilience of fungi, resulting in a potentially more powerful optimization algorithm.

Concept: Ant-Fungi Hybrid Optimization Algorithm (AFHO)
To design an algorithm that blends the strengths of both ACO and fungi-inspired methods, let's outline how this can be achieved:

### Ant-like Exploration and Pheromone Trails:

We'll maintain the classic ACO behavior where artificial ants traverse a network and deposit pheromones to communicate with each other, reinforcing the shortest paths.
Ants will prioritize paths with higher pheromone levels but also explore new paths with a certain probability, balancing exploitation and exploration.

### Fungi-like Network Adaptation:

Introduce a mycelium-inspired adaptive network structure where nodes (representing possible paths) can grow, shrink, or retract based on their utility.
Use a feedback mechanism similar to how fungi reinforce nutrient-rich paths and retract from inefficient ones.
Incorporate dynamic path adjustment that allows for the network to change in real-time, adapting to new conditions or obstacles.

### Hybrid Pheromone and Nutrient Flow:
Instead of just relying on pheromones, introduce a "nutrient flow" concept similar to how fungi transport nutrients through their network.
Paths that are frequently used by ants receive both pheromone reinforcement and nutrient flow, allowing for adaptive reinforcement.

### Key Benefits of the Hybrid Approach Speed and Adaptability: 
Combines the rapid path-finding capabilities of ants with the adaptive network resilience of fungi.
Dynamic Optimization: Adapts to changing environments and finds optimal solutions even if the landscape changes.
Scalability: Can be applied to large, complex networks like logistics, supply chain management, or transportation systems.
This approach leverages the best of both worlds to provide a robust solution for complex optimization problems.

### Pandas Generated Dataset

A pandas generated dataset with random values but ranged within a realistic parameter.
Driving distance garnered from Google Maps manually and load capacity of vehicle types placed between a reasonable range to ensure reproducability as well as variance.

This dataset is for illistration purposes only and the real dataset will be an amalgamation of several real world sources from IOT devices mounted on vehicles.

In [86]:
import pandas as pd
import numpy as np
import random

# Precomputed driving distances from Google Maps
precomputed_distances = {
    ('Bujumbura', 'Gitega'):98.9,
    ('Juba', 'Wau'):647,
    ('Nairobi', 'Arusha'):274,
    ('Kampala', 'Bujumbura'):720,
    ('Kampala', 'Juba'):653,
    ('Kampala', 'Kigali'):511,
    ('Kigali', 'Bujumbura'):273,
    ('Nairobi', 'Bujumbura'):1281,
    ('Dar es Salaam', 'Kigali'):1444,
    ('Dar es Salaam', 'Arusha'):630,
    ('Dar es Salaam', 'Nairobi'):822,
    ('Dar es Salaam', 'Bujumbura'):1496,
    ('Dar es Salaam', 'Kampala'):1501,
    ('Dar es Salaam', 'Juba'):2013,
    ('Bujumbura', 'Dar es Salaam'):1496,
    ('Bujumbura', 'Kisumu'):1064,
    ('Nairobi', 'Arusha'):902,  
    ('Nairobi', 'Dar es Salaam'):1012,  
    ('Nairobi', 'Kampala'):663,  
    ('Nairobi', 'Wau'):1868,  
    ('Nairobi', 'Kigali'):1354,
    ('Kisumu','Kampala'):324,
    ('Kisumu','Kigali'):834,
    ('Kisumu','Bujumbura'):1064,
    ('Kisumu','Juba'):841,
    ('Kisumu','Wau'):1488,
    ('Kisumu','Nairobi'):341,
    ('Lamu Port', 'Kampala'):1352,
    ('Lamu Port', 'Nairobi'):697,
    ('Lamu Port', 'Kigali'):1808,
    ('Lamu Port', 'Juba'):1849,
    ('Lamu Port', 'Bujumbura'):1868,
    ('Lamu Port', 'Kisumu'):1046,
    ('Lamu Port', 'Gitega'):1769,
    ('Lamu Port', 'Arusha'):690,
}

# Stopover cities
stopover_cities = {
    ('Nairobi', 'Dar es Salaam'): 'Arusha',
    ('Nairobi', 'Dar es Salaam'): 'Mombasa',
    ('Nairobi', 'Kampala'): 'Kisumu',
    ('Nairobi', 'Wau'): 'Juba',
    ('Nairobi', 'Kigali'): 'Arusha',
    ('Lamu Port', 'Kigali'): 'Arusha',
    ('Lamu Port', 'Bujumbura'): 'Arusha',
    ('Lamu Port', 'Gitega'): 'Arusha',
}

# Expand city pairs to generate 1000 entries
random.seed(42)
expanded_city_pairs = random.choices(list(precomputed_distances.keys()), k=1000)

# Function to safely look up stopover cities
def get_stopover(origin, destination):
    return stopover_cities.get((origin, destination)) or stopover_cities.get((destination, origin)) or None

# Generate vehicle plates and stopover info
stopovers = []
vehicle_plates = []

for i, (origin, destination) in enumerate(expanded_city_pairs):
    stopovers.append(get_stopover(origin, destination))
    vehicle_plates.append(f"{origin[:3].upper()}-{destination[:3].upper()}-{str(i).zfill(4)}")

# Create DataFrame for city pairs
sub_saharan_cities = pd.DataFrame({
    'City of Origin': [pair[0] for pair in expanded_city_pairs],
    'Destination City': [pair[1] for pair in expanded_city_pairs],
    'Stopover City': stopovers,
    'Vehicle Plate': vehicle_plates,
    'Distance (km)': [precomputed_distances[pair] for pair in expanded_city_pairs]  # Use precomputed distance
})

# Vehicle types
np.random.seed(42)
vehicle_types = ['SUV', 'Truck', 'Saloon', 'Coupe', 'Hybrid']
vehicle_type = np.random.choice(vehicle_types, size=len(vehicle_plates))

# Speed limits per vehicle type
speed_ranges = {
    'SUV': (60, 120),
    'Truck': (40, 90),
    'Saloon': (50, 110),
    'Coupe': (70, 130),
    'Hybrid': (40, 100),
}

# Fuel consumption per 100 km (L/100km)
fuel_consumption_ranges = {
    'SUV': (10, 20),
    'Truck': (20, 35),
    'Saloon': (7, 15),
    'Coupe': (8, 18),
    'Hybrid': (5, 12),
}

# Generate speed and fuel consumption values
speeds = []
fuel_consumptions = []

for vt in vehicle_type:
    speed_min, speed_max = speed_ranges[vt]
    fuel_min, fuel_max = fuel_consumption_ranges[vt]
    
    speed = np.random.uniform(speed_min, speed_max)  # Random speed within range
    fuel_efficiency = np.random.uniform(fuel_min, fuel_max)  # Fuel usage per 100km
    
    speeds.append(speed)
    fuel_consumptions.append(fuel_efficiency)

# Create vehicle DataFrame
realistic_vehicles = pd.DataFrame({
    'Vehicle Plate': vehicle_plates,
    'Vehicle Type': vehicle_type,
    'Speed (km/h)': speeds,
    'Fuel Consumption (L/100km)': fuel_consumptions,
})

# Add traffic congestion levels
traffic_congestion = np.random.choice(['Low', 'Medium', 'High'], size=len(vehicle_plates))
realistic_vehicles['Traffic Congestion'] = traffic_congestion

# Adjust speed based on congestion (+/- 20 km/h)
speed_adjustments = {'Low': +20, 'Medium': 0, 'High': -20}
realistic_vehicles['Adjusted Speed (km/h)'] = realistic_vehicles.apply(
    lambda row: max(30, row['Speed (km/h)'] + speed_adjustments[row['Traffic Congestion']]), axis=1
)

# Compute travel time in hours (distance / adjusted speed)
merged_data = sub_saharan_cities.merge(realistic_vehicles, on='Vehicle Plate')

merged_data['Travel Time (hours)'] = merged_data['Distance (km)'] / merged_data['Adjusted Speed (km/h)']

# Compute fuel consumption for full trip
merged_data['Total Fuel Consumption (L)'] = (merged_data['Distance (km)'] * merged_data['Fuel Consumption (L/100km)']) / 100

# Define peak and off-peak times based on speed variations
merged_data['Peak Time'] = merged_data['Travel Time (hours)'] * np.random.uniform(1.1, 1.3, len(merged_data))
merged_data['Off-Peak Time'] = merged_data['Travel Time (hours)'] * np.random.uniform(0.7, 0.9, len(merged_data))

# Compute Travel Time Index (TTI)
# TTI = (Travel time under congestion) / (Free-flow travel time)
merged_data['Free-Flow Speed (km/h)'] = merged_data['Vehicle Type'].map(lambda vt: speed_ranges[vt][1])
merged_data['Free-Flow Time (hours)'] = merged_data['Distance (km)'] / merged_data['Free-Flow Speed (km/h)']
merged_data['Travel Time Index (TTI)'] = merged_data['Travel Time (hours)'] / merged_data['Free-Flow Time (hours)']

# Define average payload capacities in pounds
payload_capacities = {
    'SUV': 1750,      # Average of 1,500 to 2,000 pounds
    'Truck': 1600,    # Example from Honda Ridgeline
    'Saloon': 900,    # Average of 800 to 1,000 pounds
    'Coupe': 800,
    'Hybrid': 1000,   # Example from Honda CR-V Hybrid
}

# Function to assign load weight based on vehicle type
def assign_load_weight(vehicle_type):
    avg_payload = payload_capacities.get(vehicle_type, 1000)  # Default to 1000 if type is unknown
    # Simulate actual load as a percentage of the payload capacity (e.g., 50% to 100%)
    load_percentage = np.random.uniform(0.5, 1.0)
    return avg_payload * load_percentage

# Apply the function to assign load weights
merged_data['Load Weight (pounds)'] = merged_data['Vehicle Type'].apply(assign_load_weight)

# Save the merged dataset
merged_data.to_csv('optimized_transport_data.csv', index=False)

# Display the first few rows
print("\nOptimized Transport Dataset (First 5 Rows):")
print(merged_data.head())



Optimized Transport Dataset (First 5 Rows):
  City of Origin Destination City Stopover City Vehicle Plate  Distance (km)  \
0         Kisumu           Kigali          None  KIS-KIG-0000          834.0   
1      Bujumbura           Gitega          None  BUJ-GIT-0001           98.9   
2  Dar es Salaam           Arusha          None  DAR-ARU-0002          630.0   
3        Nairobi        Bujumbura          None  NAI-BUJ-0003         1281.0   
4         Kisumu          Nairobi          None  KIS-NAI-0004          341.0   

  Vehicle Type  Speed (km/h)  Fuel Consumption (L/100km) Traffic Congestion  \
0        Coupe    125.927097                   13.651332               High   
1       Hybrid     81.799049                   11.457496             Medium   
2       Saloon     92.434318                    8.220312             Medium   
3       Hybrid     74.577302                    9.247005             Medium   
4       Hybrid     65.447840                   10.155110             Medium   


In [87]:
unique_origins = merged_data["City of Origin"].unique()
print(unique_origins)

['Kisumu' 'Bujumbura' 'Dar es Salaam' 'Nairobi' 'Lamu Port' 'Juba'
 'Kigali' 'Kampala']


In [88]:
unique_destinations = merged_data["Destination City"].unique()
print(unique_destinations)

['Kigali' 'Gitega' 'Arusha' 'Bujumbura' 'Nairobi' 'Juba' 'Dar es Salaam'
 'Wau' 'Kampala' 'Kisumu']


In [89]:
unique_stopover = merged_data["Stopover City"].unique()
print(unique_stopover)

[None 'Arusha' 'Kisumu' 'Juba' 'Mombasa']
